first of all, load the master dataset


In [1]:
import csv
import pyfpgrowth


In [2]:
def bin(val, interval):
    return int(int(val)/interval)

Latitude and longitude don't matter. Bin the service requests( sum of open services and closed service requests) with interval of 5- 0, 5, 10, 15 etc are the values. If 0, then between 0 to 5, if 5 then between 5 and 10 and so on

In [18]:
binned_set = []
with open('data/master.csv', 'r') as csvfile:
    masterreader = csv.reader(csvfile)
    for index, row in enumerate(masterreader):
        if index == 0: continue
        transaction_list = []
        # Add the data to binned list
        if float(row[5]) > 0.0:
            binned_sr = bin(row[5], 5)
            transaction_list.append("SERVICE_REQUESTS_" + str(binned_sr))
        
            avg_days = float(row[6])
            if avg_days < 10:
                transaction_list.append("SERVICE_DAYS_QUICK")
            elif avg_days < 30:
                transaction_list.append("SERVICE_DAYS_MED")
            else:
                transaction_list.append("SERVICE_DAYS_LATE")
            
        traffic_violations = float(row[7])
        
        if traffic_violations > 0:
            if traffic_violations < 100:
                transaction_list.append("TRAFFIC_VIOL_LOW")
            elif traffic_violations < 500:
                transaction_list.append("TRAFFIC_VIOL_MED")
            else:
                transaction_list.append("TRAFFIC_VIOL_HIGH")
            
        traffic_vol = float(row[8]) 
        if traffic_vol > 0:
            if traffic_vol < 1000:
                transaction_list.append("TRAFFIC_VOL_LOW")
            elif traffic_vol < 5000:
                transaction_list.append("TRAFFIC_VOL_MED")
            else:
                transaction_list.append("TRAFFIC_VOL_HIGH")
        
        no_crimes = float(row[9])
        if no_crimes > 0.0:
            if no_crimes < 100:
                transaction_list.append("CRIMES_LOW")
                continue
            elif no_crimes < 1000:
                transaction_list.append("CRIMES_MED")
            else:
                transaction_list.append("CRIMES_HIGH")
            
        # calculate arrest rate instead of individual arrests as more arrest happen in places with
        # more crimes
        if no_crimes > 0:
            arrests = float(row[11])
            arrest_rate = (arrests * 100)/ no_crimes
            if arrest_rate < 30:
                transaction_list.append("ARREST_RATE_LOW")
            elif arrest_rate < 1000:
                transaction_list.append("ARREST_RATE_MED")
            else:
                transaction_list.append("ARREST_RATE_HIGH")
        # police stations : if inside 50 block, near. inside 300, medium, outside: far
        police_distance = float(row[12])
        if police_distance < 50:
            transaction_list.append("POLICE_NEAR")
        elif police_distance < 300:
            transaction_list.append("POLICE_MED")
        else:
            transaction_list.append("POLICE_FAR")
            
        school_distance = float(row[12])
        if school_distance < 50:
            transaction_list.append("SCHOOL_NEAR")
        elif school_distance < 300:
            transaction_list.append("SCHOOL_MED")
        else:
            transaction_list.append("SCHOOL_FAR")
        # Same goes with schools
        binned_set.append(transaction_list)

In [19]:
sup_per = 0.02
sup = (sup_per/100.0) * len(binned_set)
patterns = pyfpgrowth.find_frequent_patterns(binned_set, sup)

In [20]:
len(patterns)

1009

In [21]:
for pat in patterns:
    if any("HIGH" in a for a in pat):
        print(pat)

('CRIMES_MED', 'SCHOOL_NEAR', 'SERVICE_REQUESTS_1', 'TRAFFIC_VIOL_HIGH', 'TRAFFIC_VOL_HIGH')
('POLICE_NEAR', 'SERVICE_REQUESTS_0', 'TRAFFIC_VIOL_HIGH', 'TRAFFIC_VOL_HIGH')
('CRIMES_MED', 'POLICE_NEAR', 'SCHOOL_NEAR', 'SERVICE_DAYS_MED', 'SERVICE_REQUESTS_0', 'TRAFFIC_VOL_HIGH')
('CRIMES_MED', 'POLICE_MED', 'TRAFFIC_VOL_HIGH')
('TRAFFIC_VIOL_MED', 'TRAFFIC_VOL_HIGH')
('ARREST_RATE_MED', 'CRIMES_MED', 'SCHOOL_NEAR', 'SERVICE_REQUESTS_0', 'TRAFFIC_VOL_HIGH')
('CRIMES_MED', 'POLICE_NEAR', 'SERVICE_DAYS_QUICK', 'SERVICE_REQUESTS_0', 'TRAFFIC_VIOL_HIGH', 'TRAFFIC_VOL_HIGH')
('ARREST_RATE_LOW', 'SCHOOL_NEAR', 'SERVICE_REQUESTS_1', 'TRAFFIC_VIOL_HIGH')
('SCHOOL_NEAR', 'SERVICE_DAYS_MED', 'TRAFFIC_VIOL_HIGH', 'TRAFFIC_VOL_HIGH')
('ARREST_RATE_LOW', 'POLICE_NEAR', 'SERVICE_DAYS_QUICK', 'SERVICE_REQUESTS_0', 'TRAFFIC_VOL_HIGH')
('CRIMES_MED', 'POLICE_NEAR', 'SCHOOL_NEAR', 'SERVICE_REQUESTS_0', 'TRAFFIC_VOL_HIGH')
('POLICE_NEAR', 'SCHOOL_NEAR', 'SERVICE_DAYS_QUICK', 'SERVICE_REQUESTS_1', 'TRAFFIC_